In [1]:
%%HTML
<link rel="stylesheet" href="https://doc.splicemachine.com/jupyter/css/custom.css">

In [ ]:
import os
os.environ['JDBC_HOST'] = 'jrtest01-splice-hregion'

# The Life of a Query

This notebook walks you through running and optimizing a query in Splice Machine, using the TPC-H benchmarking data that we imported into your database at the beginning of this class. We explore these topics:

* *Examining a Query Execution Plan*
* *Informing the Optimizer*
* *Adding Indexes to the Database*
* *A Glimpse at Splice Machine Benchmark Results*
* *Running Queries*

Note that the code in this notebook assumes that you have already loaded the TPCH-1 data with the earlier notebook run.


## Examining a Query Execution Plan

In the next few sections of this notebook, we'll examine execution plans for TPC-H Query 4, which is known as the <em>Order Priority Checking Query</em>. This query counts the number of orders ordered in a given quarter of a given year in which at least one lineitem was received by the customer later than its committed date; you can use it to determine how well the order priority system is working and gives an assessment of customer satisfaction.

Splice Machine generates an execution plan prior to running your query. You can use the `explain` command to generate and display the execution plan without actually running the query; this can help you to determine optimizing strategies for your queries. 
<p class="noteIcon">The <a href="https://doc.splicemachine.com/developers_tuning_explainplan_examples.html" target="_blank">Reading Explain Plans</a> topic in our documentation describes how to read explain plans.</p>


### Dropping Statistics

You'll recall that we collected statistics when we loaded the TPCH-1 data.  But what if we hadn't?  We can see the effect of this by first dropping the statistics:

In [ ]:
%%sql 

CALL SYSCS_UTIL.DROP_SCHEMA_STATISTICS('DEV2');


Now, when we run explain on Query 4 we'll see that the row counts are back to approximated estimates. For example, recall that the `LINEITEM` table has about 6M rows;  compare that to the bottom `scannedRows` count in the explain output:


In [ ]:
%%sql 
-- QUERY 04
explain  select
	o_orderpriority,
	count(*) as order_count
from
	DEV2.orders
where
	o_orderdate >= date('1993-07-01')
	and o_orderdate < add_months('1993-07-01',3)
	and exists (
		select
			*
		from
			DEV2.lineitem
		where
			l_orderkey = o_orderkey
			and l_commitdate < l_receiptdate
	)
group by
	o_orderpriority
order by
	o_orderpriority
-- END OF QUERY

## Optimizing Query Performance

In this section we'll look at optimizing the execution plan for TPCH Query 4. We'll:

* Collect Statistics to Inform the Optimizer
* Add Indexes to Further Optimize the Plan
* Compare Execution Plans

When creating a plan for a query, our optimizer performs a number of important and valuable actions, including:

* It creates an access plan, which determine the best path for accessing the data the query will operate upon; for example, the access path might be to scan an entire table or to use an index.
* When joining tables, the optimizer evaluates the best *join order* and the *join strategy* to use.
* The optimizer unrolls subqueries to reduce processing time

Since there often are different options available (whether or not to use an index, which join order, etc.), we evaluate the different possibilities, score them, then choose the best we find.  Of course coming up with good scores requires good knowledge about your database, and that's where the statistics collection comes in.

You use our `analyze` command to collect statistics from your database, which the optimizer uses when planning the execution of a query.

<p class="noteIcon">Cost-based optimizers are powerful features of modern databases that enable query plans to change as the data profiles change. Optimizers make use of count distinct, quantiles, and most frequent item counts as heuristics.</p>

Collecting these metrics can be extremely expensive but if approximate results are acceptable (which is typically the case with query optimization), there is a class of specialized algorithms, called streaming algorithms, or *sketches*, that can produce results orders-of magnitude faster and with mathematically proven error bounds. Splice Machine leverages the <a href="https://datasketches.github.io/docs/TheChallenge.html" target="_blank">Yahoo Sketches Library</a> for its statistics gathering. 

### Collect Statistics
Our first optimization is to collect statistics to inform the optimizer about our database. We use our `analyze` command to collect statistics on a schema (or table). This process requires a couple minutes.


In [ ]:
%%sql 
analyze schema DEV2;


### Rerun the Explain Plan After Collecting Statistics

Now let's re-run the `explain` plan for Query 4 and see how the optimizer changed the plan after gathering statistics. Note that the `scannedRows` estimate for `LINEITEM`  is appropriately at 6M rows, etc:


In [ ]:
%%sql 
-- QUERY 04
explain select
	o_orderpriority,
	count(*) as order_count
from
	DEV2.orders
where
	o_orderdate >= date('1993-07-01')
	and o_orderdate < add_months('1993-07-01',3)
	and exists (
		select
			*
		from
			DEV2.lineitem
		where
			l_orderkey = o_orderkey
			and l_commitdate < l_receiptdate
	)
group by
	o_orderpriority
order by
	o_orderpriority
-- END OF QUERY

### Compare Execution Plans After Analyzing the Database

Now let's compare the plans to see what changed. At a quick glance, you'll notice that a very large difference in the `totalCost` numbers for every operation in the plan.  (Note 
that your exact costs will vary slightly from what we show here, depending on your system):

#### After Collecting Statistics
```
Plan
Cursor(n=13,rows=5,updateMode=READ_ONLY (1),engine=Spark)
  ->  ScrollInsensitive(n=12,totalCost=16920.058,outputRows=5,outputHeapSize=127 B,partitions=41)
    ->  OrderBy(n=11,totalCost=16919.956,outputRows=5,outputHeapSize=127 B,partitions=41)
      ->  ProjectRestrict(n=10,totalCost=16517.046,outputRows=1604125,outputHeapSize=127 B,partitions=41)
        ->  GroupBy(n=9,totalCost=3955.595,outputRows=1604125,outputHeapSize=39.081 MB,partitions=41)
          ->  ProjectRestrict(n=8,totalCost=3004,outputRows=435327,outputHeapSize=39.081 MB,partitions=41)
            ->  MergeSortJoin(n=7,totalCost=3955.595,outputRows=1604125,outputHeapSize=39.081 MB,partitions=41,preds=[(ExistsFlatSubquery-0-1.L_ORDERKEY[7:1] = O_ORDERKEY[7:2])])
              ->  TableScan[ORDERS(1616)](n=6,totalCost=3004,scannedRows=1500000,outputRows=435327,outputHeapSize=39.081 MB,partitions=41,preds=[(O_ORDERDATE[5:2] >= 1993-07-01),(O_ORDERDATE[5:2] < dataTypeServices: DATE )])
              ->  ProjectRestrict(n=5,totalCost=11385.304,outputRows=1980401,outputHeapSize=31.163 MB,partitions=41)
                ->  Distinct(n=4,totalCost=277.69,outputRows=1501009,outputHeapSize=23.619 MB,partitions=1)
                  ->  ProjectRestrict(n=3,totalCost=11385.304,outputRows=1980401,outputHeapSize=31.163 MB,partitions=41)
                    ->  ProjectRestrict(n=2,totalCost=11385.304,outputRows=1980401,outputHeapSize=31.163 MB,partitions=41,preds=[(L_COMMITDATE[0:2] < L_RECEIPTDATE[0:3])])
                      ->  TableScan[LINEITEM(1600)](n=1,totalCost=11286.284,scannedRows=6001215,outputRows=6001215,outputHeapSize=31.163 MB,partitions=41)
```

#### Before Collecting Statistics
```
Plan
Cursor(n=13,rows=36753750,updateMode=READ_ONLY (1),engine=Spark)
  ->  ScrollInsensitive(n=12,totalCost=1576165.614,outputRows=36753750,outputHeapSize=139.022 MB,partitions=41)
    ->  OrderBy(n=11,totalCost=838921.423,outputRows=36753750,outputHeapSize=139.022 MB,partitions=41)
      ->  ProjectRestrict(n=10,totalCost=400696.835,outputRows=36753750,outputHeapSize=139.022 MB,partitions=41)
        ->  GroupBy(n=9,totalCost=98186.501,outputRows=36753750,outputHeapSize=139.022 MB,partitions=41)
          ->  ProjectRestrict(n=8,totalCost=75629,outputRows=11837789,outputHeapSize=139.022 MB,partitions=41)
            ->  MergeSortJoin(n=7,totalCost=98186.501,outputRows=36753750,outputHeapSize=139.022 MB,partitions=41,preds=[(ExistsFlatSubquery-0-1.L_ORDERKEY[7:1] = O_ORDERKEY[7:2])])
              ->  TableScan[ORDERS(1616)](n=6,totalCost=75629,scannedRows=37812500,outputRows=11837789,outputHeapSize=139.022 MB,partitions=41,preds=[(O_ORDERDATE[5:2] >= 1993-07-01),(O_ORDERDATE[5:2] < dataTypeServices: DATE )])
              ->  ProjectRestrict(n=5,totalCost=275416.5,outputRows=45375000,outputHeapSize=129.819 MB,partitions=41)
                ->  Distinct(n=4,totalCost=6717.476,outputRows=45375000,outputHeapSize=129.819 MB,partitions=1)
                  ->  ProjectRestrict(n=3,totalCost=275416.5,outputRows=45375000,outputHeapSize=129.819 MB,partitions=41)
                    ->  ProjectRestrict(n=2,totalCost=275416.5,outputRows=45375000,outputHeapSize=129.819 MB,partitions=41,preds=[(L_COMMITDATE[0:2] < L_RECEIPTDATE[0:3])])
                      ->  TableScan[LINEITEM(1600)](n=1,totalCost=275004,scannedRows=137500000,outputRows=137500000,outputHeapSize=129.819 MB,partitions=41)
```


### Optimize by Adding Indexes

Splice Machine tables have primary keys either implicit or explicitly defined. Data is stored in order of these keys.

<div class="noteNote" style="max-width:40%;">The primary key is not optimal for all queries.</div>

Unlike HBase and other key-value stores, Splice Machine can use *secondary indexes* to improve the performance of data manipulation statements. In addition, `UNIQUE` indexes provide a form of data integrity checking.

In Splice Machine, an index is just another HBase table, keyed on the index itself.

### Adding an index on ORDERS

Note that in the explain for this query, we are scanning the entire `ORDERS` table, even though we only will require a subset of the data. Adding an index on `O_ORDERDATE` should help. HOWEVER, it's important to know that the plan is telling us that even if we use an index, we still will be returning many rows to the next step (`outputRows>400K`).   This means that we should be careful to avoid the `IndexLookup` problem that we previously discussed, and that we should also add other columns that we'll need.


In [ ]:
%%sql 

create index DEV2.O_DATE_PRI_KEY_IDX on DEV2.ORDERS(
 O_ORDERDATE,
 O_ORDERPRIORITY,
 O_ORDERKEY
 );


Here's the query again on which to rerun `explain`, so we can compare plans.


In [ ]:
%%sql 
-- QUERY 04
explain select
	o_orderpriority,
	count(*) as order_count
from
	DEV2.orders
where
	o_orderdate >= date('1993-07-01')
	and o_orderdate < add_months('1993-07-01',3)
	and exists (
		select
			*
		from
			DEV2.lineitem
		where
			l_orderkey = o_orderkey
			and l_commitdate < l_receiptdate
	)
group by
	o_orderpriority
order by
	o_orderpriority
-- END OF QUERY

### Compare Updated Execution Plans

We can now compare how the query will execute with indexing in place versus without indexes. You'll again notice that, among other differences, the `totalCost` values are lower for most operations because the optimizer was able to take advantage of the indexes we added.

#### Query Plan After Indexing
```
Plan
Cursor(n=14,rows=5,updateMode=READ_ONLY (1),engine=Spark)
  ->  ScrollInsensitive(n=13,totalCost=15353.855,outputRows=5,outputHeapSize=127 B,partitions=41)
    ->  OrderBy(n=12,totalCost=15353.753,outputRows=5,outputHeapSize=127 B,partitions=41)
      ->  ProjectRestrict(n=11,totalCost=14988.138,outputRows=1604125,outputHeapSize=127 B,partitions=41)
        ->  GroupBy(n=10,totalCost=2463.089,outputRows=1604125,outputHeapSize=39.081 MB,partitions=41)
          ->  ProjectRestrict(n=9,totalCost=6001884,outputRows=435332,outputHeapSize=39.081 MB,partitions=41)
            ->  MergeSortJoin(n=8,totalCost=2463.089,outputRows=1604125,outputHeapSize=39.081 MB,partitions=41,preds=[(ExistsFlatSubquery-0-1.L_ORDERKEY[7:1] = O_ORDERKEY[7:2])])
              ->  ProjectRestrict(n=7,totalCost=6001884,outputRows=435332,outputHeapSize=39.081 MB,partitions=41)
                ->  IndexScan[O_DATE_PRI_KEY_IDX(1745)](n=6,totalCost=1547.029,scannedRows=1160172,outputRows=435332,outputHeapSize=39.081 MB,partitions=41,baseTable=ORDERS(1616),preds=[(O_ORDERDATE[5:1] < dataTypeServices: DATE ),(O_ORDERDATE[5:1] >= 1993-07-01)])
              ->  ProjectRestrict(n=5,totalCost=11385.304,outputRows=1980401,outputHeapSize=31.163 MB,partitions=41)
                ->  Distinct(n=4,totalCost=277.69,outputRows=1501009,outputHeapSize=23.619 MB,partitions=1)
                  ->  ProjectRestrict(n=3,totalCost=11385.304,outputRows=1980401,outputHeapSize=31.163 MB,partitions=41)
                    ->  ProjectRestrict(n=2,totalCost=11385.304,outputRows=1980401,outputHeapSize=31.163 MB,partitions=41,preds=[(L_COMMITDATE[0:2] < L_RECEIPTDATE[0:3])])
                      ->  TableScan[LINEITEM(1600)](n=1,totalCost=11286.284,scannedRows=6001215,outputRows=6001215,outputHeapSize=31.163 MB,partitions=41)
```

#### Query Plan Before Indexing
```
Plan
Cursor(n=13,rows=5,updateMode=READ_ONLY (1),engine=Spark)
  ->  ScrollInsensitive(n=12,totalCost=16920.058,outputRows=5,outputHeapSize=127 B,partitions=41)
    ->  OrderBy(n=11,totalCost=16919.956,outputRows=5,outputHeapSize=127 B,partitions=41)
      ->  ProjectRestrict(n=10,totalCost=16517.046,outputRows=1604125,outputHeapSize=127 B,partitions=41)
        ->  GroupBy(n=9,totalCost=3955.595,outputRows=1604125,outputHeapSize=39.081 MB,partitions=41)
          ->  ProjectRestrict(n=8,totalCost=3004,outputRows=435327,outputHeapSize=39.081 MB,partitions=41)
            ->  MergeSortJoin(n=7,totalCost=3955.595,outputRows=1604125,outputHeapSize=39.081 MB,partitions=41,preds=[(ExistsFlatSubquery-0-1.L_ORDERKEY[7:1] = O_ORDERKEY[7:2])])
              ->  TableScan[ORDERS(1616)](n=6,totalCost=3004,scannedRows=1500000,outputRows=435327,outputHeapSize=39.081 MB,partitions=41,preds=[(O_ORDERDATE[5:2] >= 1993-07-01),(O_ORDERDATE[5:2] < dataTypeServices: DATE )])
              ->  ProjectRestrict(n=5,totalCost=11385.304,outputRows=1980401,outputHeapSize=31.163 MB,partitions=41)
                ->  Distinct(n=4,totalCost=277.69,outputRows=1501009,outputHeapSize=23.619 MB,partitions=1)
                  ->  ProjectRestrict(n=3,totalCost=11385.304,outputRows=1980401,outputHeapSize=31.163 MB,partitions=41)
                    ->  ProjectRestrict(n=2,totalCost=11385.304,outputRows=1980401,outputHeapSize=31.163 MB,partitions=41,preds=[(L_COMMITDATE[0:2] < L_RECEIPTDATE[0:3])])
                      ->  TableScan[LINEITEM(1600)](n=1,totalCost=11286.284,scannedRows=6001215,outputRows=6001215,outputHeapSize=31.163 MB,partitions=41)
```

### Run Query 4

Now go ahead and run TPC-H Query 4.  Again feel free to use the Console (at `localhost:4040`) to monitor the query while it runs.

## A Glimpse at Splice Machine Benchmark Results

Here are some micro-benchmark results from Splice Machine running TPC-H and other benchmarks:

- 2ms single record lookups on primary keys at petabyte scale
- 20ms single record updates at petabyte scale
- 40-way OLTP indexed joins return in <100ms
- 150-way OLAP style joins execute in under 2 minutes
- 440-way join executes where others can’t parse
- Ingestion at 80MB/sec/node
- Can run TPC-C and TPC-H simultaneously


## Where to Go Next

The next notebook in this class shows you how you can [*Visualize Results with Jupyter*](./e.%20Transactions%20with%20Spark%20and%20JDBC.ipynb).